# 1. Import Libraries Required

In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import warnings

warnings.filterwarnings("ignore")

/Users/ardiansyahputra/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 2. Loading Files and Model

In [2]:
with open('list_num_cols.txt', 'r') as file_1:
    list_num_cols = json.load(file_1)

with open('scaler.pkl', 'rb') as file_2:
    scaler = pickle.load(file_2)

with open('pca.pkl', 'rb') as file_3:
    pca = pickle.load(file_3)

with open('model_km.pkl', 'rb') as file_4:
    km = pickle.load(file_4)


# 3. Preparing Data in order to conduct Inference

In [28]:
data_inf = {
    'cust_id': [4455, 4456, 4457, 4458, 4459, 4460, 4461, 4462, 4463, 4464, 4465, 4466, 4467, 4468, 4469, 4470, 4471, 4472, 4473, 4474],
    'balance': [4872, 5562, 6228, 8136, 276, 4842, 4718, 6094, 980, 7240, 1588, 6900, 2528, 1226, 2152, 4630, 8614, 4296, 4476, 3564],
    'balance_frequency': [1338.934027, 1130.827049, 951.675294, 1360.795674, 3570.596193, 1735.137321, 1108.316745, 851.880648, 392.896462, 4372.395867, 538.768762, 7468.012010, 427.642111, 706.327578, 1420.727073, 1079.008102, 809.582978, 3386.490883, 4840.545574, 1977.007090],
    'purchases': [1.0, 0.909091, 1.0, 1.0, 1.0, 1.0, 1.0, 0.909091, 0.909091, 1.0, 0.909091, 1.0, 0.888889, 0.888889, 1.0, 1.0, 0.909091, 1.0, 1.0, 1.0],
    'oneoff_purchases': [0.00, 0.00, 0.00, 0.00, 150.46, 1879.92, 480.70, 1476.70, 450.00, 569.91, 226.95, 1569.41, 0.00, 169.54, 0.00, 0.00, 0.00, 560.98, 1595.93, 1032.04],
    'installments_purchases': [0.00, 0.00, 0.00, 0.00, 150.46, 0.00, 480.70, 542.11, 0.00, 0.00, 97.37, 773.34, 0.00, 0.00, 0.00, 0.00, 0.00, 133.76, 0.00, 805.14],
    'cash_advance': [1297.777178, 1727.256593, 1090.520215, 462.861443, 3381.913668, 934.051432, 2856.358168, 903.935644, 517.197717, 520.844673, 452.009130, 4500.288676, 2563.705644, 1406.735573, 311.804499, 1259.571764, 3819.672466, 2008.153534, 4359.325750, 4335.919227],
    'purchases_frequency': [0.0, 0.0, 0.0, 0.0, 0.181818, 0.454545, 0.909091, 0.636364, 0.818182, 0.272727, 0.545455, 0.545455, 0.0, 0.333333, 0.0, 0.0, 0.0, 1.0, 0.454545, 0.090909],
    'oneoff_purchases_frequency': [0.0, 0.0, 0.0, 0.0, 0.0, 0.454545, 0.0, 0.454545, 0.818182, 0.272727, 0.090909, 0.454545, 0.0, 0.333333, 0.0, 0.0, 0.0, 0.727273, 0.454545, 0.090909],
    'purchases_installments_frequency': [0.0, 0.0, 0.0, 0.0, 0.090909, 0.0, 0.909091, 0.363636, 0.0, 0.0, 0.454545, 0.363636, 0.0, 0.0, 0.0, 0.0, 0.0, 0.363636, 0.0, 0.818182],
    'cash_advance_frequency': [0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.272727, 0.888889, 0.888889, 0.900000, 0.545455, 0.545455, 0.545455, 0.545455, 0.545455],
    'cash_advance_trx': [7, 7, 3, 6, 7, 3, 11, 3, 5, 9, 8, 7, 62, 14, 9, 9, 7, 9, 17, 21],
    'purchases_trx': [0, 0, 0, 0, 2, 8, 10, 18, 3, 5, 9, 10, 0, 3, 0, 0, 0, 20, 25, 17],
    'credit_limit': [4000.0, 1500.0, 1200.0, 1500.0, 6500.0, 4000.0, 2500.0, 1200.0, 500.0, 4500.0, 1200.0, 8700.0, 1200.0, 2500.0, 1500.0, 1200.0, 4150.0, 10100.0, 6000.0, 4000.0],
    'payments': [423.092478, 1897.193158, 178.362607, 303.031979, 1233.301038, 3986.076109, 2884.208351, 2207.649199, 769.083695, 1345.099869, 946.649324, 2162.943350, 2853.967334, 302.976780, 506.765033, 1379.635287, 4372.148648, 1039.000037, 905.844846, 3056.424974],
    'minimum_payments': [634.989453, 490.973854, 340.410195, 443.781316, 911.267314, 429.538814, 413.570374, 613.130977, 496.215787, 3937.465141, 211.220001, 4628.802111, 146.976542, 142.597254, 449.179818, 271.364415, 241.645387, 886.050237, 2376.919568, 910.707525],
    'prc_full_payment': [0.0, 0.090909, 0.0, 0.0, 0.0, 0.090909, 0.181818, 0.125000, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.090909, 0.090909, 0.0, 0.090909, 0.090909],
    'tenure': [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 9, 9, 10, 11, 11, 11, 11, 11]
}


In [14]:
list_num_cols

['balance',
 'purchases',
 'oneoff_purchases',
 'installments_purchases',
 'cash_advance',
 'purchases_frequency',
 'oneoff_purchases_frequency',
 'purchases_installments_frequency',
 'cash_advance_frequency',
 'cash_advance_trx',
 'purchases_trx',
 'credit_limit',
 'payments',
 'minimum_payments',
 'prc_full_payment']

In [29]:
data_inf

{'cust_id': [4455,
  4456,
  4457,
  4458,
  4459,
  4460,
  4461,
  4462,
  4463,
  4464,
  4465,
  4466,
  4467,
  4468,
  4469,
  4470,
  4471,
  4472,
  4473,
  4474],
 'balance': [4872,
  5562,
  6228,
  8136,
  276,
  4842,
  4718,
  6094,
  980,
  7240,
  1588,
  6900,
  2528,
  1226,
  2152,
  4630,
  8614,
  4296,
  4476,
  3564],
 'balance_frequency': [1338.934027,
  1130.827049,
  951.675294,
  1360.795674,
  3570.596193,
  1735.137321,
  1108.316745,
  851.880648,
  392.896462,
  4372.395867,
  538.768762,
  7468.01201,
  427.642111,
  706.327578,
  1420.727073,
  1079.008102,
  809.582978,
  3386.490883,
  4840.545574,
  1977.00709],
 'purchases': [1.0,
  0.909091,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.909091,
  0.909091,
  1.0,
  0.909091,
  1.0,
  0.888889,
  0.888889,
  1.0,
  1.0,
  0.909091,
  1.0,
  1.0,
  1.0],
 'oneoff_purchases': [0.0,
  0.0,
  0.0,
  0.0,
  150.46,
  1879.92,
  480.7,
  1476.7,
  450.0,
  569.91,
  226.95,
  1569.41,
  0.0,
  169.54,
  0.0,
  0.

In [30]:
data_inf_df = pd.DataFrame(data_inf)

In [31]:
data_inf_df

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
0,4455,4872,1338.934027,1.000000,0.00,0.00,1297.777178,0.000000,0.000000,0.000000,0.272727,7,0,4000.0,423.092478,634.989453,0.000000,11
1,4456,5562,1130.827049,0.909091,0.00,0.00,1727.256593,0.000000,0.000000,0.000000,0.272727,7,0,1500.0,1897.193158,490.973854,0.090909,11
2,4457,6228,951.675294,1.000000,0.00,0.00,1090.520215,0.000000,0.000000,0.000000,0.272727,3,0,1200.0,178.362607,340.410195,0.000000,11
3,4458,8136,1360.795674,1.000000,0.00,0.00,462.861443,0.000000,0.000000,0.000000,0.272727,6,0,1500.0,303.031979,443.781316,0.000000,11
4,4459,276,3570.596193,1.000000,150.46,150.46,3381.913668,0.181818,0.000000,0.090909,0.272727,7,2,6500.0,1233.301038,911.267314,0.000000,11
5,4460,4842,1735.137321,1.000000,1879.92,0.00,934.051432,0.454545,0.454545,0.000000,0.272727,3,8,4000.0,3986.076109,429.538814,0.090909,11
6,4461,4718,1108.316745,1.000000,480.70,480.70,2856.358168,0.909091,0.000000,0.909091,0.272727,11,10,2500.0,2884.208351,413.570374,0.181818,11
7,4462,6094,851.880648,0.909091,1476.70,542.11,903.935644,0.636364,0.454545,0.363636,0.272727,3,18,1200.0,2207.649199,613.130977,0.125000,11
8,4463,980,392.896462,0.909091,450.00,0.00,517.197717,0.818182,0.818182,0.000000,0.272727,5,3,500.0,769.083695,496.215787,0.000000,11
9,4464,7240,4372.395867,1.000000,569.91,0.00,520.844673,0.272727,0.272727,0.000000,0.272727,9,5,4500.0,1345.099869,3937.465141,0.000000,11


# 4. Model's Prediction

In [32]:
data_inf_df[list_num_cols]

,balance,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment
0,4872,1.000000,0.00,0.00,1297.777178,0.000000,0.000000,0.000000,0.272727,7,0,4000.0,423.092478,634.989453,0.000000
1,5562,0.909091,0.00,0.00,1727.256593,0.000000,0.000000,0.000000,0.272727,7,0,1500.0,1897.193158,490.973854,0.090909
2,6228,1.000000,0.00,0.00,1090.520215,0.000000,0.000000,0.000000,0.272727,3,0,1200.0,178.362607,340.410195,0.000000
3,8136,1.000000,0.00,0.00,462.861443,0.000000,0.000000,0.000000,0.272727,6,0,1500.0,303.031979,443.781316,0.000000
4,276,1.000000,150.46,150.46,3381.913668,0.181818,0.000000,0.090909,0.272727,7,2,6500.0,1233.301038,911.267314,0.000000
5,4842,1.000000,1879.92,0.00,934.051432,0.454545,0.454545,0.000000,0.272727,3,8,4000.0,3986.076109,429.538814,0.090909
6,4718,1.000000,480.70,480.70,2856.358168,0.909091,0.000000,0.909091,0.272727,11,10,2500.0,2884.208351,413.570374,0.181818
7,6094,0.909091,1476.70,542.11,903.935644,0.636364,0.454545,0.363636,0.272727,3,18,1200.0,2207.649199,613.130977,0.125000
8,980,0.909091,450.00,0.00,517.197717,0.818182,0.818182,0.000000,0.272727,5,3,500.0,769.083695,496.215787,0.000000
9,7240,1.000000,569.91,0.00,520.844673,0.272727,0.272727,0.000000,0.272727,9,5,4500.0,1345.099869,3937.465141,0.000000


## Scalling

In [33]:
df_scaled = scaler.fit_transform(data_inf_df[list_num_cols])
df_scaled

array([[ 0.18153755,  0.72882079, -0.81355088, -0.56852496, -0.45042054,
        -0.94376315, -0.7945398 , -0.60574378, -0.71886298, -0.33717814,
        -0.84444074,  0.2299833 , -0.9984693 , -0.25897768, -0.59665029],
       [ 0.47564628, -1.22920521, -0.81355088, -0.56852496, -0.147079  ,
        -0.94376315, -0.7945398 , -0.60574378, -0.71886298, -0.33717814,
        -0.84444074, -0.72430424,  0.21334384, -0.37781383,  0.20556822],
       [ 0.75952514,  0.72882079, -0.81355088, -0.56852496, -0.59680625,
        -0.94376315, -0.7945398 , -0.60574378, -0.71886298, -0.65830017,
        -0.84444074, -0.83881875, -1.19965425, -0.50205319, -0.59665029],
       [ 1.57279971,  0.72882079, -0.81355088, -0.56852496, -1.04012197,
        -0.94376315, -0.7945398 , -0.60574378, -0.71886298, -0.41745864,
        -0.84444074, -0.72430424, -1.09716737, -0.4167553 , -0.59665029],
       [-1.77748233,  0.72882079, -0.57265271,  0.00501649,  1.02160617,
        -0.39399874, -0.7945398 , -0.27831491, 

## PCA

In [38]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
num_features = np.argmax(cumsum >= 0.95) + 1
num_features

9

In [39]:
pca.fit(df_scaled)
df_scaled_pca = pca.transform(df_scaled)

## Predict Cluster With Model (K-Means)

In [40]:
pred = km.fit_predict(df_scaled_pca)
pred

array([2, 2, 2, 2, 2, 3, 3, 3, 1, 2, 1, 3, 0, 1, 2, 2, 2, 3, 3, 3],
      dtype=int32)

In [41]:
data_inf_df['cluster'] = pred
data_inf_df

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure,cluster
0,4455,4872,1338.934027,1.000000,0.00,0.00,1297.777178,0.000000,0.000000,0.000000,0.272727,7,0,4000.0,423.092478,634.989453,0.000000,11,2
1,4456,5562,1130.827049,0.909091,0.00,0.00,1727.256593,0.000000,0.000000,0.000000,0.272727,7,0,1500.0,1897.193158,490.973854,0.090909,11,2
2,4457,6228,951.675294,1.000000,0.00,0.00,1090.520215,0.000000,0.000000,0.000000,0.272727,3,0,1200.0,178.362607,340.410195,0.000000,11,2
3,4458,8136,1360.795674,1.000000,0.00,0.00,462.861443,0.000000,0.000000,0.000000,0.272727,6,0,1500.0,303.031979,443.781316,0.000000,11,2
4,4459,276,3570.596193,1.000000,150.46,150.46,3381.913668,0.181818,0.000000,0.090909,0.272727,7,2,6500.0,1233.301038,911.267314,0.000000,11,2
5,4460,4842,1735.137321,1.000000,1879.92,0.00,934.051432,0.454545,0.454545,0.000000,0.272727,3,8,4000.0,3986.076109,429.538814,0.090909,11,3
6,4461,4718,1108.316745,1.000000,480.70,480.70,2856.358168,0.909091,0.000000,0.909091,0.272727,11,10,2500.0,2884.208351,413.570374,0.181818,11,3
7,4462,6094,851.880648,0.909091,1476.70,542.11,903.935644,0.636364,0.454545,0.363636,0.272727,3,18,1200.0,2207.649199,613.130977,0.125000,11,3
8,4463,980,392.896462,0.909091,450.00,0.00,517.197717,0.818182,0.818182,0.000000,0.272727,5,3,500.0,769.083695,496.215787,0.000000,11,1
9,4464,7240,4372.395867,1.000000,569.91,0.00,520.844673,0.272727,0.272727,0.000000,0.272727,9,5,4500.0,1345.099869,3937.465141,0.000000,11,2
